In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time


In [2]:
def extract_car_info(car):
    car_info = {}
    name = car.find("span", class_="car-name")
    ul_tag = car.find('ul', class_='tag-list')
    ul_info = car.find('ul', class_='seller-info verify')

    if name:
        car_info['Tên'] = name.text

    if ul_tag:
        items = ul_tag.find_all('li')
        if len(items) >= 4:
            car_info['Hộp số'] = items[0].text
            car_info['Tình trạng'] = items[1].text
            car_info['Nhiên liệu'] = items[2].text
            car_info['Odo'] = items[3].text

    price = car.find("p", class_="price")
    if price:
        car_info['Giá tiền'] = price.text

    if ul_info:
        seller_name = ul_info.find('li', class_='seller-name')
        seller_location = ul_info.find('li', class_='seller-location')
        seller_phone = ul_info.find('li', class_='seller-phone')
        if seller_name:
            seller_name_text = seller_name.find('span', class_='txt')
            if seller_name_text:
                car_info['Nơi bán'] = seller_name_text.text.strip()

        if seller_location:
            location_text = seller_location.text.strip()
            car_info['Tỉnh'] = location_text

        if seller_phone:
            phone_text = seller_phone.find('span', class_='txt')
            if phone_text:
                car_info['Số điện thoại'] = phone_text.text.strip()
    return car_info  # Trả về dict thông tin của xe



# Khởi tạo trình duyệt
driver = webdriver.Chrome()
driver.get("https://oto.com.vn/mua-ban-xe")

scrolls = 200 # Số lần scroll
car_data = []  # Khai báo danh sách lưu trữ thông tin các xe

for i in range(scrolls):
    # Scroll xuống cuối trang
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # Lấy dữ liệu sau khi đã scroll
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    car_lists = soup.find_all("div", class_="item-car vipprior dev-item-car")

    # Lặp qua danh sách các xe và trích xuất thông tin
    for car in car_lists:
        car_info = extract_car_info(car)
        car_data.append(car_info)

driver.quit()

# Tạo DataFrame từ danh sách thông tin các xe
df = pd.DataFrame(car_data, columns=['Tên', 'Tình trạng', 'Hộp số', 'Nhiên liệu', 'Odo', 'Giá tiền', 'Nơi bán', 'Tỉnh', 'Số điện thoại'])

# Xuất ra file Excel
df.to_excel('car_data.xlsx', index=False)
# Xuất ra file CSV
df.to_csv('car_data.csv', index=False)
